In [2]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../optimized/data/wiggly_banana_preopt/deployed_opt.msh')
lflat = elastic_rods.RodLinkage('../optimized/data/wiggly_banana_preopt/flat_opt.msh')
mat = elastic_rods.RodMaterial('+', 20000, 0.3, [5.0, 5.0, 0.2, 0.2], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)
lflat.setMaterial(mat)

l.setPerSegmentRestLength(np.loadtxt('../optimized/data/wiggly_banana_preopt/design_parameters.txt'))
lflat.setPerSegmentRestLength(np.loadtxt('../optimized/data/wiggly_banana_preopt/design_parameters.txt'))

jdo = l.dofOffsetForJoint(l.centralJoint())
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint

with suppress_stdout(): elastic_rods.compute_equilibrium(lflat, 0.108574, fixedVars=fixedVars)
with suppress_stdout(): elastic_rods.compute_equilibrium(l, np.deg2rad(80), fixedVars=fixedVars)
    
view = LinkageViewer(l, width=1024)
view.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [3]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

lopt = elastic_rods.RodLinkage('../optimized/data/wiggly_banana_5/deployed_opt.msh')
loptFlat = elastic_rods.RodLinkage('../optimized/data/wiggly_banana_5/flat_opt.msh')
lopt.setMaterial(mat)
loptFlat.setMaterial(mat)

lopt.setPerSegmentRestLength(np.loadtxt('../optimized/data/wiggly_banana_5/design_parameters.txt'))
loptFlat.setPerSegmentRestLength(np.loadtxt('../optimized/data/wiggly_banana_5/design_parameters.txt'))

with suppress_stdout(): elastic_rods.compute_equilibrium(loptFlat, 0.108574, fixedVars=fixedVars)
with suppress_stdout(): elastic_rods.compute_equilibrium(lopt, np.deg2rad(80), fixedVars=fixedVars)
    
viewopt = LinkageViewer(lopt, width=1024)
viewopt.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
viewopt.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [10]:
stressMax = max(max(max(l.rodStresses()[1])), max(max(lopt.rodStresses()[1])))

In [14]:
import write_render_files
import os
deploySeconds = 10
framesPerSecond = 30
frames = deploySeconds * framesPerSecond
outdir = 'wiggly_stress_frames'
try: os.mkdir(outdir)
except: pass
externalForcesPreopt = l.gradient()
externalForcesOpt = lopt.gradient()
for i, forceScale in enumerate(np.linspace(1.0, 0.0, frames)):
    framedir = "{}/{}".format(outdir, i)
    with suppress_stdout(): elastic_rods.compute_equilibrium(l, forceScale * externalForcesPreopt, fixedVars=fixedVars)
    with suppress_stdout(): elastic_rods.compute_equilibrium(lopt, forceScale * externalForcesOpt, fixedVars=fixedVars)
    write_render_files.writeRenderFiles(l, framedir, 'wiggly_preopt', stressMin = 0, stressMax = stressMax)
    write_render_files.writeRenderFiles(l, framedir, 'wiggly_opt', stressMin = 0, stressMax = stressMax)